In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Libraris to import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler, PolynomialFeatures


from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV 
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
train = pd.read_csv("/kaggle/input/new-data-pp/train_crosssell_Preprocessed.csv")
test = pd.read_csv("/kaggle/input/new-data-pp/test_crosssell_Preprocessed.csv")

In [ ]:
train_x = train.drop(columns=["id", "Vintage", "Driving_License","source","Previously_Insured","Vehicle_Damage"])
train_y = train['Response']

In [ ]:
test_x = test.drop(columns=["id",  "Vintage", "Driving_License","source","Previously_Insured","Vehicle_Damage"])


In [ ]:
one = train_x[train_x['Response']==1]
zero = train_x[train_x['Response']==0]

In [ ]:
selected_zero = zero.sample(n = 46685, random_state= 20)

In [ ]:
train_x_new = pd.concat([one,selected_zero ])

In [ ]:
train_y_new = train_x_new['Response']
train_x_new = train_x_new.drop(columns=['Response'])


In [ ]:
train_x = train_x.drop(columns=['Response'])

In [ ]:
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = train_x.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(train_x.values, i) 
                          for i in range(len(train_x.columns))] 
  
print(vif_data)

# **Modelling**

In [ ]:
def run_mod(model,train_x,train_y):
    ss = StandardScaler()
    train_x_ss = ss.fit_transform(train_x)
    model.fit(train_x_ss,train_y)
    print(model)
    #print(roc_auc_score(train_y,  model.predict_proba(train_x)[:,1]))
    print(roc_auc_score(train_y,  model.predict(train_x_ss)))
    return
    

In [ ]:
model = [AdaBoostClassifier(),RandomForestClassifier(),XGBClassifier(),LogisticRegression(),
         KNeighborsClassifier(),LGBMClassifier(),
         GaussianNB()
         ]


for i in model:
    run_mod(i,train_x_new,train_y_new)
    

In [ ]:
#gausian nb hyper parameter tuning
# defining parameter range 
param_grid = {'max_depth': [2,3,4,6,8,10,12]
              }
  
grid = GridSearchCV(RandomForestClassifier(), param_grid,verbose=1, scoring='roc_auc') 
  
# fitting the model for grid search 
grid.fit(train_x_new, train_y_new) 


# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 



In [ ]:
#Stacking

In [ ]:
ss = StandardScaler()
train_x_ss = ss.fit_transform(train_x_new)


estimator = [] 
estimator.append(('RF', RandomForestClassifier('max_depth': 10) )) 
estimator.append(('GN', GaussianNB(var_smoothing=0.006579332246575682) )) 
estimator.append(('XGB', XGBClassifier() )) 


# Voting Classifier with hard voting 
vot_hard = VotingClassifier(estimators = estimator, voting ='hard') 
vot_hard.fit(train_x_ss, train_y_new) 
#y_pred = vot_hard.predict(X_test)
print(roc_auc_score(train_y_new,  vot_hard.predict(train_x_ss)))



# Voting Classifier with soft voting 
vot_soft = VotingClassifier(estimators = estimator, voting ='soft') 
vot_soft.fit(train_x_ss, train_y_new) 
#y_pred = vot_soft.predict(X_test) 
print(roc_auc_score(train_y_new,  vot_soft.predict(train_x_ss)))




In [ ]:
test_x_ss = ss.fit_transform(test_x)
res = vot_soft.predict(test_x_ss)

In [ ]:
'''train_x_ss = ss.fit_transform(train_x_new)
test_x_ss = ss.fit_transform(test_x)
model.fit(train_x_ss,train_y_new)
res = model.predict(test_x_ss)
'''

In [ ]:
res = pd.DataFrame(res)

In [ ]:
res['id'] = test['id']

In [ ]:
res = res[['id',0]]

In [ ]:
res = res.rename(columns={0: "Response"})

In [ ]:
res.to_csv('result.csv',index = False)

In [ ]:
test.shape

In [ ]:
res.shape